# 컴퓨터 파워 UP

## 1. 들어가며

### 학습 목표
- 멀티태스킹, 병렬프로그래밍과 동시에 대해 이해
- 파이썬으로 멀티스레드와 멀티프로세스를 구현하는 법에 대해 학습
- 파이썬의 'concurrent.futures'를 이용한 병렬 프로그래밍을 구현해 보기

### 학습 내용
**1. 멀티태스킹**
- 멀티태스킹이란?
- 컴퓨터의 세계 - 프로그램
- 프로그램 실행과 프로파일링
- scale-up vs scale-out

**2. 멀티프로세스, 멀티스레드**
- 멀티스레드
- 멀티프로세스
- 스레드/프로세스 풀

**3. 실전 예제**
- 'concurrent.fetures' 모듈
- 소수 구하기 문제

### 유의사항
- 이번 노드는 파이썬 3.7.9 버전을 기준으로 작성되었음을 유의
- 파이썬 3.8 이후에는 멀티스레드 활용 등에 이 노드에서 소개하는 것과 달라진 부분이 있을 수 있음

## 2. 멀티태스킹 (1) 멀티태스킹이란?

- 빅데이터를 다루기 위해서는 몇 가지 중요한 개념들을 알아야 함
- 컴퓨팅 자원을 최적화하는 방법, 즉 병렬 컴퓨팅, 분산 컴퓨팅 등의 개념
- 어떻게 컴퓨터 파워를 향상시킬 수 있을까?
- 어떻게 한정된 컴퓨팅 자원을 활용하여 여러 가지 일을 효율적으로 진행할 수 있을까?
- 이런 고민은 비단 컴퓨터 아키텍처를 설계하는 사람들만의 전유물은 아님

- 이번에 새로 라면집을 개업한 김사장님도 비슷한 고민을 하고 있음
- 주방장 1명이 라면 1개를 끓이는 데 시간은 아래와 같이 총 10분이 소요됨
    - 물 끓이기: 3분
    - 재료 손질: 2분
    - 재료 넣고 끓이기: 4분
    - 그릇에 담아 서빙: 1분
![멀티태스킹1.png](./images/멀티태스킹1.png)
- 현재는 이렇게 라면 1개를 끓이는 과정을 완료한 후, 다음 라면을 끓이기 시작
- 그런데 주방장이 아무리 쉴 새 없이 일해도 1시간에 라면을 6개밖에 끓이지 못함
- 이래가지고는 도저히 가게 수지를 맞춰낼 수가 없음
- 이에 2가지 아이디어를 생각해냄 -> **동시성**과 **병렬성**
- 아래에 이어지는 비유와 함께 아래 링크의 글을 함께 참고하면 동시성, 병렬성을 이해하는 데 도움이 됨
- [Concurrency Vs.Parallelism](http://tutorials.jenkov.com/java-concurrency/concurrency-vs-parallelism.html)

### 동시성, 병렬성
- 멀티태스킹에는 2가지 동작 방식이 있음. 바로 동시성과 병렬성

### 동시성(Concurrency)
- 왜 주방장이 동시에 라면을 1개씩밖에 못 끓이는지. 불판이 4개나 있는데, 동시에 라면을 3~4개씩 끓여도 되는 것이 아닌가
- 생각해 보니 재료를 준비하거나 서빙을 하는 시간 3분은 몰라도, 물을 끓이거나 라면을 끓이는데 소요되는 7분 동안에는 얼마든지 다른 라면을 끓이기 위한 작업을 할 수 있을 것
- 이렇게 하나의 processor가 여러 가지 task를 동시에 수행하는 개념을 동시성이라고 함
- 실제로 그 processor는 특정 순간에는 1가지 task만을 수행하겠지만, 물을 끓이는 것처럼 다른 task를 수행할 수 있는 시간에는 task를 전환해서 효율적으로 여러 개의 task를 동시에 수행하는 것처럼 보이는 것
![동시성.png](./images/동시성.png)

### 병렬성(Parallelism)
- 물론 주방장을 1명 더 고용할 수도 있음. 주방장을 1명 더 고용한다면 최소한 라면 나오는 속도는 지금의 2배가 됨
- 라면을 끓이는 것처럼 유사한 task를 여러 processor가 동시에 수행하는 것을 병렬성이라고 함
- processor의 개수만큼 퍼포먼스가 좋아짐

![동시성2.png](./images/동시성2.png)
- 주방장 1명이 라면을 동시에 여러 개 끓이도록 하는 동시성 적용해 보았음
- 그 결과 이전보다 3배 이상 주방장의 퍼포먼스가 좋아짐
- 매출이 올랐고, 가게의 인기가 더 좋아짐
- 이에 주방장을 1명 더 고용해 보았음
> 동시성과 병렬성을 동시에 적용하면 어떻게 될까?
- 그랬더니 놀라운 일이 일어남
- 라면 1개를 끓이는데 실제로는 10분보다 훨씬 짧은 시간이 소요될 수도 있다는 것을 알게됨
- 주방장 1명이 물을 끓이는 동안 다른 주방장이 그 라면의 재료를 미리 손질해 두는 식으로, 라면 1개를 끓이는데 소요되는 시간조차 더 줄어들면서 훨씬 많은 라면을 끓여낼 수 있게 됨
![동시성_병렬성.png](./images/동시성_병렬성.png)

- 이렇듯, 병렬성의 효율을 극대화하는 것은 동시성이 요구될 때
- 이때, 여러 개의 프로세스가 1개의 task를 여러 개의 subtask로 쪼개어 동시에 병렬적으로 수행할 수 있기 때문

### 동기 vs 비동기(Synchronous vs Asynchronous)
- 동시성에서 주로 다루게 될ㄷ 개념
- 이렇게 어떤 일을 바로 하지 못하고 대기해야 하는 일을 일컬을 때 컴퓨터에서는 "바운드(bound) 되었다"라는 표현을 많이 사용
- 이렇게 바운드되고 있으면 이걸 계속 기다려야 할지 아니면 종료되는 사이에 다른 걸 실행하는 것이 좋을지 고민이 됨
- 개념을 일반화시켜 이야기하면 작동하는 방식에 따라 앞 작업이 종료되기를 무조건 기다렸다가 다음 작업을 수행하는 것은 동기(synchronized) 방식이라고 이야기함
- 기다리는 동안 다른 일을 처리하는 것을 비동기(asynchronous)방식이라고 이야기함
![동기_비동기.png](./images/동기_비동기.png)
- [그림 출처](https://poiemaweb.com/js-async)
    - 동기(Synchronous): 어떤 일이 순차적으로 실행됨. 요청과 요청에 대한 응답이 동시에 실행됨(따라서 요청에 지연이 발생하더라도 계속 대기)
    - 비동기(Asynchronous): 어떤 일이 비순차적으로 실행됨. 요청과 요청에 대한 응답이 동시에 실행되지 않음. 특정 코드의 연산이 끝날 때까지 코드의 실행을 멈추지 않고 다음 코드를 먼저 실행하며, 중간에 실행되는 코드는 주로 콜백함수로 연결되기도 함

- 위의 예에서 보자면, 물을 끓이거나 조리를 하는 과정을 생각해 볼 수 있음
- 주방장은 물을 끓이면서 굳이 다 끓을 때까지 기다릴 필요가 없음
- 하지만 물이 끓지 않으면 조리를 할 수는 없을 테니 물이 끓기 전까지는 어쩔수 없이 다음 과정을 수행하지 못하고 대기해야 할 것
- 물이 끓기 전까지 기다렸다가 재료 손질 등 다음 단계를 진행한다면 물 끓이기 작업을 동기화 방식으로 수행한 것
- 하지만, 물을 끓이기 위해 불만 켜두고는 다 끓을 때까지 다르 작업을 진행한다면 물 끓이기 작업을 비동기화 방식으로 수행한 것

### I/O Bound vs CPU Bound
- 컴퓨터가 일을 수행하면서 뭔가 기다릴 때, 즉 속도에 제한이 걸릴 때는 2가지 경우에 해당하는 경우가 대부분
    - I/O 바운드: 입력과 출력에서의 데이터(파일) 처리에 시간이 소요될 때
    - CPU 바운드: 복잡한 수식 계산이나 그래픽 작업과 같은 엄청난 계산이 필요햘 때
- 더 자세히 알고 싶다면 아래 글을 참고
- [What do the terms "CPU bound" and "I/O bound" mean?](https://stackoverflow.com/questions/868568/what-do-the-terms-cpu-bound-and-i-o-bound-mean)

## 3. 멀티태스킹 (2) 프로세스, 쓰레드, 프로파일링

- PPT 문서 작업을 하면서, 노래를 듣기도 하고, 웹 브라우저를 이용해 검색을 하기도 함
- 요즘 컴퓨터에서는 이렇게 여러 개 프로그램을 동시에 실행시키는 멀티태스킹이 기본 사양
- 맥(Mac)의 '활성 상태 보기(Activity Monitor)'나 윈도우(Windows)의 '작업 관리자(Task Manaer)'를 통해 현재 실행되고 있는 프로그램의 상태를 확인할 수 있음
![멀티태스킹.png](./images/멀티태스킹.png)
- 프로그램을 실행할 때 컴퓨터에서는 어떤 일이 일어날까?

### Process(프로세스)
> An Instance of a program (ex.Python Interpreter)
- 하나의 프로그램을 실행할 때, 운영체제는 한 프로세스를 생성함
- 프로세스는 운영체제의 커널(Kernel)에서 시스템 자원(CPU, 메모리, 디스크) 및 자료구조를 이용함
- 프로세스는 "프로그램을 구동하여 프로그램 자체와 프로그램의 상태가 메모리상에서 실행되는 작업 단위"를 지칭(출처: [위키백과 - 프로세스](https://ko.wikipedia.org/wiki/%ED%94%84%EB%A1%9C%EC%84%B8%EC%8A%A4))
- 예를 들어, 하나의 프로그램을 1번 구동하면 하나의 프로세스가 메모리상에서 실행되지만 여러 번 구동하면 여러 개의 프로세스가 실행됨
- 파이썬에서는 'os'라는 모듈에서 프로세스 관련 정보를 얻을 수 있음

In [2]:
import os

# process ID
print(os.getpid())

# user ID
print(os.getuid())

# group ID
print(os.getgid())

# 현재 작업중인 디렉토리
print(os.getcwd())

29
0
0
/aiffel/aiffel/LMS/25. 컴퓨터 파워 UP(멀티태스킹, 병렬프로그래밍, 멀티프로세스 구현)


- 방금 'os.getpid()'를 통해 얻은 'pid'는 어떤 프로세스의 것인가?
- 터미널을 열어 다음과 같이 해당 프로세스의 상세 정보 확인해 보기
![터미널.png](./images/터미널.png)
- AIFFEL LMS가 연결한 jupyter 커널 프로세스
- (참고) 만약 'ps' 명령어나 'pid' 개념이 생소하다고 느껴진다면 **터미널로 배우는 리눅스 운영체제** 노드의 **10. 리눅스 기반 운영체제 (2) 프로세스 관리와 시스템 콜** 스텝 참고

### Thread(스레드)
- 프로그램 내에서도 많은 작업이 있음
- 스레드(thread)는 어떠한 프로그램 내에서, 특히 프로세스 내에서 실행되는 흐름의 단위
- 예를 들어 요리를 만드는 프로그램이라고 한다면 김밥, 떡볶이를 만드는 각각의 요리라는 프로세스에도 밥 짓기, 재료 볶기, 끓이기 등등의 작업을 스레드에 비유할 수 있음
- 같은 작업을 좀 더 빠르게 처리하기 위해 여러 개의 스레드를 생성하기도 함
- 프로세스 vs 스레드의 차이가 명확하게 이해가 가는가?
- 아래 그림을 예로 들자면, 프로세스는 김밥, 떡볶이를 만드는 각각의 요리사와 같음
- 이들은 각각의 전용 주방 공간에서 밥 짓기, 재료 볶기, 끓이기 등등의 작업을 병렬적으로 수행
- 도마, 불판 등 주방 공간은 각각의 작업에 공유되지만, 요리사끼리 주방공간을 공유하지는 않음
- 마찬가지로 프로세스도 자신만의 전용 메모리 공간(Heap)을 가짐
- 이때 해당 프로세스 내의 스레드들은 이 메모리 공간을 공유
- 그러나 다른 프로세스와 공유하지는 않음
- 스레드의 사용은 프로그램마다 다르지만, 가벼운 프로그램은 하나의 스레드를 가지기도 함
![프로세스_스레드.png](./images/프로세스_스레드.png)
![프로세스.png](./images/프로세스.png)
- [그림 출처](https://en.wikipedia.org/wiki/Thread_(computing))

### 프로파일링(Profiling)
- 코드에서 시스템의 어느 부분이 느린지 혹은 어디서 RAM을 많이 사용하고 있는지를 확인하고 싶을 때 사용하는 기법
- 맥(mac)의 활성 상태 보기(activity monitor)나 윈도우(windows)의 작업 관리자(task manager)를 통해 현재 실행 중인 프로그램의 상태를 확인하는 작업을 코딩하는 것을 프로파일링이라고 함
- 파이썬에서는 다음 명령어로 확인할 수 있음
![프로파일링.png](./images/프로파일링.png)
- 이렇게 간단하게 프로그램의 수행 시간을 계산해 보면서 프로파일링 할 수 있음
- 아래 예제 살펴보며 100까지 숫자를 배열로 만드는 방법을 각각 다른 방법으로 구현해 보고 실행 시간을 측정해 보기

In [3]:
import timeit
        
def f1():
    s = set(range(100))

    
def f2():
    l = list(range(100))

    
def f3():
    t = tuple(range(100))


def f4():
    s = str(range(100))

    
def f5():
    s = set()
    for i in range(100):
        s.add(i)

def f6():
    l = []
    for i in range(100):
        l.append(i)
    
def f7():
    s_comp = {i for i in range(100)}

    
def f8():
    l_comp = [i for i in range(100)]
    

if __name__ == "__main__":
    t1 = timeit.Timer("f1()", "from __main__ import f1")
    t2 = timeit.Timer("f2()", "from __main__ import f2")
    t3 = timeit.Timer("f3()", "from __main__ import f3")
    t4 = timeit.Timer("f4()", "from __main__ import f4")
    t5 = timeit.Timer("f5()", "from __main__ import f5")
    t6 = timeit.Timer("f6()", "from __main__ import f6")
    t7 = timeit.Timer("f7()", "from __main__ import f7")
    t8 = timeit.Timer("f8()", "from __main__ import f8")
    print("set               :", t1.timeit(), '[ms]')
    print("list              :", t2.timeit(), '[ms]')
    print("tuple             :", t3.timeit(), '[ms]')
    print("string            :", t4.timeit(), '[ms]')
    print("set_add           :", t5.timeit(), '[ms]')
    print("list_append       :", t6.timeit(), '[ms]')
    print("set_comprehension :", t5.timeit(), '[ms]')
    print("list_comprehension:", t6.timeit(), '[ms]')

set               : 1.9519889885559678 [ms]
list              : 0.9095842959359288 [ms]
tuple             : 0.8529953379184008 [ms]
string            : 0.5633771913126111 [ms]
set_add           : 6.263241769745946 [ms]
list_append       : 5.5437213480472565 [ms]
set_comprehension : 6.301398925483227 [ms]
list_comprehension: 5.642286446876824 [ms]


- 이런 식으로 함수의 성능을 측정할 수 있음
- 사실 이건 시간을 측정하는 방법이라서 프로파일링이라고 하기에는 좀 민망할 수 있음
> 좀 더 엄밀히 말하면 **프로파일링**은 애플리케이션에서 가장 자원이 집중되는 지점을 정밀하게 찾아내는 기법<br>
> **프로파일러**는 애플리케이션을 실행시키고 함수 실행에 드는 시간을 찾아내는 프로그램<br>
> 즉, **코드의 병목(bottleneck)** 을 찾아내고 **성능을 측정**해 주는 도구
- 'profle' 모듈, [cProfile](https://docs.python.org/ko/3/library/profile.html) 모듈과 [line_profiler](https://frhyme.github.io/python-libs/python_line_profileing_in_python/) 패키지를 이용하면 높은 수준의 프로파일링이 가능한 점 참고

## 4. 멀티태스킹 (3) Scale Up vs Scale Out

- 프로그램과 성능에 대한 전반적인 개념 살펴보았음
- 더 포괄적인 용어로 말하면 우리는 컴퓨터 자원을 활용하기 위해 자원을 Up(업그레이드, 최적화)시킬 수도 있고 자원을 Out(확장)시킬 수도 있음
- Scale-Up은 1대의 컴퓨터의 성능을 최적화시키는 방밥이고 Scale-Out은 여러 대의 컴퓨터를 1대처럼 사용하는 것
![scale_up.png](./images/scale_up.png)
- 상세 내용은 이 [블로그](https://hyuntaeknote.tistory.com/m/4) 참고
> 멀티 태스킹, 프로그램, 동시성과 병렬성, 동시성에서 나오게 될 개념인 비동기와 동기, 추가로 바운드에 대한 설명<br>
> 처음에 멀티 태스킹의 개념을 사람에 비유했다면, 이제는 프로그램에서의 용어로 풀어서 설명<br>
> 프로세스와 스레드, 프로파일링, 그리고 마지막으로 Scale-Up와 Scale-Out이라는 단어들이 나왔음<br>
> 직접 구현해 가면서 성능이 어떻게 최적화되는지 위 개념에 대해 천천히 배워보기

## 5. 파이썬에서 멀티스레드 사용하기 (1) 스레드 생성

- 파이썬에서 멀티스레드의 구현은 'threading' 모듈 이용
- [threading - Thread-based parallelism - Python 3.7.9 documentation](https://docs.python.org/3.7/library/threading.html)

### 기본 코드
- 아래 코드는 음식 배달과 그릇 찾기 2가지 작업을 **순차적**으로 수행하는 코드

In [4]:
class Delivery:
    def run(self):
        print("delivery")
        
class RetriveDish:
    def run(self):
        print("Retriving Dish")
        
work1 = Delivery()
work2 = RetriveDish()

def main():
    work1.run()
    work2.run()
    
if __name__ == '__main__':
    main()

delivery
Retriving Dish


### 멀티스레드
- 다음과 같이 코드 고쳐보기
    - threading 모듈을 import하고
    - 클래스에 Thread 상속받기

In [5]:
from threading import *

class Delivery(Thread):
    def run(self):
        print("delivery")
        
class RetriveDish(Thread):
    def run(self):
        print("Retriving Dish")
        
work1 = Delivery()
work2 = RetriveDish()

def main():
    work1.run()
    work2.run()
    
if __name__ == '__main__':
    main()

delivery
Retriving Dish


### 스레드 생성 확인
- 함수 이름을 출력하면 함수 객체를 확인할 수 있음

In [6]:
from threading import *

class Delivery:
    def run(self):
        print("delivering")
        
work1 = Delivery()
print(work1.run)

class Delivery(Thread):
    def run(self):
        print("delivering")
        
work2 = Delivery()
print(work2.run)

<bound method Delivery.run of <__main__.Delivery object at 0x7f11a41edbd0>>
<bound method Delivery.run of <Delivery(Thread-6, initial)>>


- 위 코드를 실행해 확인해 보면, 'work2'는 다른 스레드(Thread-6, 컴퓨터마다 다름)에 할당됨

## 6. 파이썬에서 멀티스레드 사용하기 (2) 스레드 생성 및 사용

### 스레드 생성
- 'threading' 모듈의 'Thread' 클래스를 상속받아서 구현할 수도 있지만 그대로 인스턴스화하여 스레드를 생성할 수도 있음
- 인스턴스화 하려면 'Thread' 클래스에 인자로 'target'과 'args' 값 넣어주기
- 'args'에 넣어 준 파라미터는 스레드 함수의 인자로 넘어감
![스레드.png](./images/스레드.png)
- Thread 클래스에는 start(), join() 같은 스레드 동작 관련 메소드가 있음
- Thread로 실행할 함수를 정의한 후 start()를 통해 스레드 실행

In [8]:
from threading import *
from time import sleep

Stopped = False

# 일꾼 스레드
def worker(work, sleep_sec):
    # 그만하라고 할때까지
    while not Stopped:
        # 시키는 일을 하고
        print('do', work)
        # 잠깐 쉬기
        sleep(sleep_sec)
        # 언젠가 이 굴레를 벗어나면, 은퇴할 때가 올 것?
    print('retired..')

# 일꾼 스레드를 하나 생성. 열심히 일하고 3초간 쉬기
t = Thread(target = worker, args = ('Overwork', 3))
t.start()

# 일꾼, 이제 일을 해야지? 😈
t.start()

do Overwork


RuntimeError: threads can only be started once

do Overwork
do Overwork
do Overwork


- 이 일꾼의 일을 어떻게 멈춰주면 좋을까?
- t.terminate() 같은 메소드가 있느냐 하면, 없음
- 그래스 스레드 함수가 루프를 돌  때는 꼭 멈춰야 할지를 체크하는 flag(여기서는 Stopped)를 체크하도록 설계
    - (주의) 이 예제에서 while 문의 실행 결과를 제대로 보려면 별도의 jupyter notebook 화면에서 실행하기를 권함

In [9]:
# 이 코드 블록을 실행하기 전까지는 일꾼 스레드는 종료하지 않음

# 일꾼 일 그만하라고 세팅
Stopped = True

# 일꾼 스레드가 종료할때까지 기다림
t.join()
print('worker is gone.')

retired..
worker is gone.
retired..


## 7. 파이썬에서 멀티프로세스 사용하기

- 파이썬에서 멀티프로세서의 구현은 'multiprocessing' 모듈을 이용해서 할 수 있음
- [multiprocessing - Process-based parallelism - Python 3.7.9 documentation](https://docs.python.org/3.7/library/multiprocessing.html)

### 프로세스 생성
- 프로세스 또한 스레드와 유사한 방법으로 생성
    - 'Process' 인스턴스를 만든 뒤, 'target'과 'args' 파라미터에 각각 함수 이름과 함수 인자를 전달

In [10]:
import multiprocessing as mp

def delivery():
    print('delivering...')
    
p = mp.Process(target = delivery, args = ())
p.start()

delivering...


### 프로세스 사용
- 'Process' 클래스는 'start()', 'join()', 'terminate()' 같은 프로세스 동작 관련 메소드가 있음
![프로세스사용.png](./images/프로세스사용.png)

## 8. 파이썬에서 스레드/프로세스 풀 사용하기

- 지금까지 스레드와 프로세스 생성하는 방법 배웠음
- 사실 멀티스레드/프로세스 작업을 할 때 가장 많은 연산이 필요한 작업은 바로 이런 스레드나 프로세스를 생성하고 종료하는 일
- 특히 스레드/프로세스를 사용한 뒤에는 제대로 종료해 주어야 컴퓨팅 리소스가 낭비되지 않음
- 지금까지 본 것처럼 하나씩 하나씩 실행한다고 전체적인 프로그램의 성능이 좋아지지는 않음
- 오히려 더 번거로울 수 있음
- 그래서 실제로 사용할 때에는 스레드/프로세스 풀을 사용해서 생성
- "풀"이란 영어로 'Pool'. 스레드나 프로세스들로 가득 찬 풀장
- 스레드 풀을 만들면 각각의 테스크들에 대해 자동으로 스레드들을 할당하고 종료
- 풀을 만드는 방법은 크게 2가지가 있음
    - [Quene](https://docs.python.org/3.7/library/queue.html)를 사용해서 직접 만드는 방법
    - [concurrent.futures]() 라이브러리의 'ThreadPoolExecutor', 'ProcessPoolExecutor' 클래스를 이용하는 방법
- 2가지 방법 모두 많이 사용하는 방법
- 아직 자료구조에 대해 배우지 않았으니 여기서는 'concurrent.futures' 모듈을 사용하는 방법만을 이용해 구현해보기

### 1. concurrent.future 모듈 소개
- 이 모듈은 파이썬 3.2부터 추가된 모듈
- Java 언어를 배운 적이 있다면, 'ThreadPoolExecutor'에 대해 들어본 적 있을 것
- 이를 파이썬에서 구현한 형태라고 생각하면 편함
- 한국말로 "동시성 퓨처"라고 번역해서 부르기도 하는데, 기능은 크게 4가지고 있음
    - Executor: 객체
    - ThreadPoolExecutor: 객체
    - ProcessPoolExecutor: 객체
    - Future: 객체
- 여기서는 Thread/Process Pool 구현에 필요한 부분만 살펴볼것

#### ThreadPoolExecutor
- 'Executor' 객체를 이용하면 스레드 생성, 시작, 조인 같은 작업을 할 때, 'with' 컨텍스트 관리자와 같은 방법으로 가독성 높은 코드를 구현할 수 있음(프로세스 구현 방법 역시 동일하므로 설명은 스레드로만 진행)
![ThreadPoolExecutor.png](./images/ThreadPoolExecutor.png)

In [11]:
# 앞서 한 'Delivery' 클래스를 예시로 들면 다음과 같음
from concurrent.futures import ThreadPoolExecutor

class Delivery:
    def run(self):
        print("delivering")
w = Delivery()

with ThreadPoolExecutor() as executor:
    future = executor.submit(w.run)

delivering


#### multiprocessing.Pool
- multiprocessing.Pool.map을 통해 여러 개의 프로세스에 특정 함수를 매핑해서 병렬처리하도록 구현하는 방법이 널리 사용되고 있음

In [12]:
from multiprocessing import Pool
from os import getpid

def double(i):
    # pool 안에서 이 메소드가 실행할 때, pid를 확인해보기
    print("I'm processing", getpid())
    return i * 2

with Pool() as pool:
    result = pool.map(double, [1, 2, 3, 4, 5])
    print(result)

I'm processingI'm processingI'm processingI'm processingI'm processing  77 
8078 

81 
79
[2, 4, 6, 8, 10]


- 'double(i)'라는 메소드가 pool을 통해 각각 다른 pid를 가진 프로세스들 위에서 multiprocess로 실행되었다는 것을 확인할 수 있음
- 프로그래밍이 처음인 분에게는 조금 낯선 개념일 수 있음
- 다음 스텝에서 실제 프로젝트로 구현해볼 예정

## 9. 실전 예제 (1) futures 모듈

- 'concurrent.futures' 모듈의 'ProcessPoolExecutor'를 이용해 병렬 프로그래밍을 연습해 보기
- **concurrent.futures 모듈 공식 문서 읽어보기**
- [concurrent.futures - Launching parallel tasks - Python 3.7.9 documentation](https://docs.python.org/ko/3.7/library/concurrent.futures.html)

- Q1. concurrent.futres 모듈에는 2가지 객체가 있음
- A1. Executor, Futures
- Q2. Executor 객체에는 2개의 서브클래스가 있음
- A2. ThreadPoolExecutor, ProcessPoolExecutor
- Q3. Executor 객체가 제공하는 3개의 메소드
- A3. submit(), map(), shutdown()

## 10. 실전 예제 (2) 튜토리얼

- 예제에서 다루는 부분은 아래 2개
    - Executor 객체의 map() 함수
    - ProcessPoolExecutor 부분
- 'concurrent.futures' 모듈의 'ProcessPoolExecutor'를 이용해서 멀티프로세스를 구현하는 연습
- 아래 코드는 [concurrent.futures 모듈 공식 문서](https://docs.python.org/ko/3.7/library/concurrent.futures.html)의 *ProcessPoolExecutor* 예제에 실린 코드


In [13]:
import math
import concurrent

PRIMES = [
    112272535095293,
    112582705942171,
    112272535095293,
    115280095190773,
    115797848077099,
    1099726899285419
]

def is_prime(n):
    if n < 2:
        return False
    if n == 2:
        return True
    if n % 2 == 0:
        return False
    
    sqrt_n = int(math.floor(math.sqrt(n)))
    for i in range(3, sqrt_n + 1, 2):
        if n % 1 == 0:
            return Fale
    return True

def main():
    with concurrent.futures.ProcessPoolExecutor() as executor:
        for number, prime in zip(PRIMES, executor.map(is_prime, PRIMES)):
            print('%d is prime: %s' % (number, prime))
            
if __name__ == '__main__':
    main

- 위 코드를 하나씩 뜯어보기

### 1) 문제
- 소수(prime) 판별 문제로 PRIMES 변수에 선언된 숫자들이 소수인지 아닌지를 판별

In [14]:
import math
import concurrent

PRIMES = [
    112272535095293,
    112582705942171,
    112272535095293,
    115280095190773,
    115797848077099,
    1099726899285419]

print("*    .\n·   *\n  *   *\n🌲 🦕 🌳")

*    .
·   *
  *   *
🌲 🦕 🌳


### 2) 소수판별 함수 is_prime
- 소수를 판별하는 로직을 함수로 구현

In [15]:
def is_prime(n):
    if n < 2:
        return False
    if n == 2:
        return True
    if n % 2 == 0:
        return False

    sqrt_n = int(math.floor(math.sqrt(n)))
    for i in range(3, sqrt_n + 1, 2):
        if n % i == 0:
            return False
    return True
print("🌲      🦕...")

🌲      🦕...


### 3) 소수 판별 함수 호출
- main() 함수를 따로 작성하여 소수 판별 함수(is_prime)를 호출
- 맵-리듀스(map-reduce) 스타일로 코드를 작성하고 map()함수를 ProcessPoolExecutor() 인스턴스에서 생성된 executor에서 실행시킴
- concurrent.futures 라이브러리의 프로세스 풀에서 동작하게 하기 위해 'with' 문을 써서 구현

In [16]:
def main():
    with concurrent.futures.ProcessPoolExecutor() as executor:
        for number, prime in zip(PRIMES, executor.map(is_prime, PRIMES)):
            print('%d is prime: %s' % (number, prime))
print("🌲   🦕.....")

🌲   🦕.....


- 병럴처리와 단일처리의 비교를 위해 코드를 아래와 같이 수정
    - 프로파일링을 위한 시간 계산 코드를 추가
    - 단일처리로 수행했을 때의 코드를 추가. 단일처리 프로그래밍을 위한 시간 계산 코드를 추가

In [17]:
import time

def main():
    print("병렬처리 시작")
    start = time.time()
    with concurrent.futures.ProcessPoolExecutor() as executor:
        for number, prime in zip(PRIMES, executor.map(is_prime, PRIMES)):
            print('%d is prime: %s' % (number, prime))
    end = time.time()
    print("병렬처리 수행 시각", end-start, 's')

    start = time.time()
    for number, prime in zip(PRIMES, map(is_prime, PRIMES)):
        print('%d is prime: %s' % (number, prime))
    end = time.time()
    print("단일처리 수행 시각", end-start, 's')
print(" ❣\n🌲🦕.......")

 ❣
🌲🦕.......


### 4) 전체 코드

In [18]:
main()

병렬처리 시작
112272535095293 is prime: True
112582705942171 is prime: True
112272535095293 is prime: True
115280095190773 is prime: True
115797848077099 is prime: True
1099726899285419 is prime: False
병렬처리 수행 시각 0.7449173927307129 s
112272535095293 is prime: True
112582705942171 is prime: True
112272535095293 is prime: True
115280095190773 is prime: True
115797848077099 is prime: True
1099726899285419 is prime: False
단일처리 수행 시각 2.808274030685425 s


- 수행 시간은 PC 성능에 따라 다르지만, 2.6GHz 6코어를 사용하는 CPU에서 돌린 결과, 병렬로 처리하면 약 0.7초가 걸리고 단일처리하면 2.8초가 걸림